# Imports

## Environment Variables

In [6]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
ANTHROPIC_MODEL = os.getenv("ANTHROPIC_MODEL")
OPENAI_MODEL = os.getenv("OPENAI_MODEL")

# Verify keys are loaded
print(f'Anthropic API Key loaded: {bool(os.getenv("ANTHROPIC_API_KEY"))}\nAnthropic model: {os.getenv("ANTHROPIC_MODEL")}')
print(f'OpenAI API Key loaded: {bool(os.getenv("OPENAI_API_KEY"))}\nOpenAI model: {os.getenv("OPENAI_MODEL")}')
print(OPENAI_MODEL)

Anthropic API Key loaded: True
Anthropic model: claude-sonnet-4-20250514
OpenAI API Key loaded: True
OpenAI model: gpt-4o-mini
gpt-4o-mini


## Packages

In [7]:
from typing import Literal, TypedDict, Optional
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver

# Configure LLM
&rarr; choose anthropic / openai 

In [8]:
def get_llm(provider: Literal["anthropic", "openai"] = "openai"):
    """Get LLM instance based on provider choice"""
    if provider == "anthropic":
        return ChatAnthropic(
            model=ANTHROPIC_MODEL,
            temperature=0.7,
            max_tokens=1024
        )
    elif provider == "openai":
        return ChatOpenAI(
            model=OPENAI_MODEL,
            temperature=0.7,
            max_tokens=1024
        )
    else:
        raise ValueError(f"Unknown provider: {provider}")

# Choose your provider here
LLM_PROVIDER = "openai"  # Change to "anthropic" if you prefer
llm = get_llm(LLM_PROVIDER)

print(f"Using {LLM_PROVIDER.upper()} as LLM provider")

Using OPENAI as LLM provider


# States 

In [9]:
# State = the data that flows through the workflow

class State(TypedDict):
    messages: list                  # Conversation history
    name: str                       # Employee name
    start_date: str                 # Start date
    end_date: str                   # End date (optional)
    work_hours: float               # Weekly work hours
    salary: float                   # Salary
    info_complete: bool             # All info collected?
    human_decision: str             # "approve" or "reject"
    is_update: bool                 # New entry or update existing?

# Nodes
NB: Chatsbots are nodes

In [ ]:
# NODE 1: Chatbot - collects employee info
def chatbot(state: State) -> State:
    """Talks to user, collects: name, start_date, end_date, work_hours, salary"""
    
    # TODO: Add LLM conversation logic here
    # - Check what info is missing
    # - Ask user for missing info
    # - Parse response and update state
    
    print("💬 [chatbot] Collecting employee information...")
    return state

In [ ]:
# NODE 2: Human verification - PAUSES for human to review
def human_verification(state: State) -> State:
    """Shows collected data, waits for human to approve or reject"""
    
    print("⏸️  [human_verification] Please review the data:")
    print(f"    Name: {state.get('name')}")
    print(f"    Start: {state.get('start_date')}")
    print(f"    End: {state.get('end_date')}")
    print(f"    Hours: {state.get('work_hours')}")
    print(f"    Salary: {state.get('salary')}")
    
    return state

In [ ]:
# NODE 3: Create new entry
def create_entry(state: State) -> State:
    """Creates a new employee record"""
    
    # TODO: Add database insert logic here
    print(f"✅ [create_entry] Created new entry for: {state.get('name')}")
    return state


# NODE 4: Update existing entry  
def update_entry(state: State) -> State:
    """Updates an existing employee record"""
    
    # TODO: Add database update logic here
    print(f"✅ [update_entry] Updated entry for: {state.get('name')}")
    return state

In [ ]:
# ROUTER 1: After chatbot - keep collecting or go to verification?
def route_after_chatbot(state: State) -> str:
    if state.get("info_complete"):
        return "human_verification"  # All info collected -> verify
    return "chatbot"                 # Missing info -> keep asking


# ROUTER 2: After human verification - what did human decide?
def route_after_verification(state: State) -> str:
    decision = state.get("human_decision")
    
    if decision == "reject":
        return "chatbot"             # Rejected -> go back to fix
    
    if decision == "approve":
        if state.get("is_update"):
            return "update_entry"    # Update existing
        return "create_entry"        # Create new
    
    return "chatbot"                 # No decision -> go back

# Workflow

In [ ]:
# Build the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("chatbot", chatbot)
workflow.add_node("human_verification", human_verification)
workflow.add_node("create_entry", create_entry)
workflow.add_node("update_entry", update_entry)

# Add edges (connections)
workflow.add_edge(START, "chatbot")                    # START -> chatbot

workflow.add_conditional_edges(                        # chatbot -> ? (depends on info_complete)
    "chatbot",
    route_after_chatbot,
    ["chatbot", "human_verification"]
)

workflow.add_conditional_edges(                        # human_verification -> ? (depends on decision)
    "human_verification", 
    route_after_verification,
    ["chatbot", "create_entry", "update_entry"]
)

workflow.add_edge("create_entry", END)                 # create_entry -> END
workflow.add_edge("update_entry", END)                 # update_entry -> END

In [ ]:
# Compile with human-in-the-loop (pauses at human_verification)
memory = MemorySaver()

app = workflow.compile(
    checkpointer=memory,
    interrupt_before=["human_verification"]  # <-- PAUSE here for human review
)

# Visualize graph

In [ ]:
# Visualize the workflow
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

# Run the workflow

In [ ]:
# Initial state (empty)
initial_state = {
    "messages": [],
    "name": "",
    "start_date": "",
    "end_date": "",
    "work_hours": 0.0,
    "salary": 0.0,
    "info_complete": False,
    "human_decision": "",
    "is_update": False,
}

# Config with thread ID (needed for checkpointing)
config = {"configurable": {"thread_id": "test-1"}}

# Run until it pauses at human_verification
for event in app.stream(initial_state, config):
    print(event)

In [ ]:
# After human reviews, update state and resume
# Example: human approves creating a new entry

app.update_state(config, {"human_decision": "approve", "is_update": False})

# Resume the workflow
for event in app.stream(None, config):
    print(event)